In [14]:
import cv2
import numpy as np
import os
 
from os.path import isfile, join
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in sorted(os.listdir(pathIn)) if isfile(join(pathIn, f))]
  
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        #print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
 
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'MJPG'), fps, size)
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
 
def main():
    folderlist = sorted(os.listdir('own_images/testing/'))
    for folder in folderlist:        
        pathIn= './own_images/testing/'+folder+'/output_lstm/'
        pathOut = './own_images/testing/'+folder+folder+'_video.avi'
        fps = 25.0
        convert_frames_to_video(pathIn, pathOut, fps)
        print('Folder '+folder+' done')
 
if __name__=="__main__":
    main()

Folder video15 done
Folder video4 done
Folder video5 done


In [27]:
arr= np.load('own_images/testing/videomix/bbox_video.npy')
arr2= np.load('own_images/testing/videomix/gt.npy')


In [12]:
# arr[244:255]

In [2]:
import cv2
import numpy as np


In [145]:
def load_yolo3_output(w_img,h_img,foldf,foldb,batch_size, num_steps, id):    
        video_frame_features = foldf
        video_frame_bboxes = np.asarray(foldb,dtype=np.float32)
        wid = w_img
        ht = h_img
        num_obj=2

        lastseen_feature=0
        lastseen_bboxes=0
        yolo_output_batched=[]
        for frame_num in range(id,id+num_steps):  # if id=2 gather frames from 2 to 7
            yolo_output_temp=[]
            for obj in range(0,num_obj):
                if  video_frame_features[frame_num][obj].all()!=0:
                    lastseen_feature=frame_num
                if  video_frame_features[frame_num][obj].all()==0:
                    video_frame_features[frame_num][obj]=video_frame_features[lastseen_feature][obj]
                    lastseen_feature=frame_num
                yoloFeat = video_frame_features[frame_num][obj].tolist()  #frame_num,obj1,255 features
                yolo_output_temp = yolo_output_temp + yoloFeat


                if  video_frame_bboxes[frame_num][obj].all()!=0:
                    lastseen_bboxes=frame_num
                if  video_frame_bboxes[frame_num][obj].all()==0:
                    video_frame_bboxes[frame_num][obj]=video_frame_bboxes[lastseen_bboxes][obj]
                    lastseen_bboxes=frame_num
                #video_frame_bboxes[frame_num][obj][2] += video_frame_bboxes[frame_num][obj][0] # convert w,h to bottom right 
                #video_frame_bboxes[frame_num][obj][3] += video_frame_bboxes[frame_num][obj][1]


                yoloBB = video_frame_bboxes[frame_num][obj].tolist() #frame_num, obj1,4 bbox attributes
                yolo_output_temp = yolo_output_temp + yoloBB
            yolo_output_batched = yolo_output_batched + yolo_output_temp

        yolo_output_batched = np.reshape(yolo_output_batched, [num_steps,batch_size,259])
        
        return yolo_output_batched

In [146]:
feat = np.load('own_images/testing/videomix/features_video.npy')
bb = np.load('own_images/testing/videomix/bbox_video.npy')

In [164]:
answer = load_yolo3_output(0,0,feat[100:150],bb[100:150],2,6,0) 
answer = np.reshape(answer,newshape=(answer.shape[1],answer.shape[0],answer.shape[2]))

In [185]:
for i in range(6):
    print(answer[0][i][255:])
answer.shape

[ 222.  119.  481.  484.]
[   4.  135.  228.  476.]
[ 223.  119.  478.  485.]
[   4.  135.  239.  480.]
[ 229.  126.  475.  480.]
[   5.  140.  239.  476.]


(2, 6, 259)

In [199]:
for i in range(7):
    print(bb[i+100][0][:])

[ 222.  119.  481.  484.]
[ 223.  119.  478.  485.]
[ 229.  126.  475.  480.]
[ 233.  127.  474.  481.]
[ 235.  131.  472.  481.]
[ 237.  124.  467.  483.]
[ 243.  130.  465.  483.]


In [190]:
def load_rolo_gt(w_img,h_img,arr,batch_size,num_steps,id):

    video_frame_bboxes = np.asarray(arr,dtype=np.float32)
    num_obj=2
    wid = w_img
    ht = h_img
    yolo_output_batched=[]
    lastseen_bboxes=0
    #frame_num= id+num_steps
    #if frame_num>772:
    #    return [0,0,0,0]
    for frame_num in range(id+num_steps,id+1+(num_steps),num_steps):
        yolo_output_temp=[]
        for obj in range(0,num_obj):
            # video_frame_bboxes[frame_num][obj][2] += video_frame_bboxes[frame_num][obj][0] # convert w,h to bottom right 
            # video_frame_bboxes[frame_num][obj][3] += video_frame_bboxes[frame_num][obj][1]

            if  video_frame_bboxes[frame_num][obj].all()!=0:
                lastseen_bboxes=frame_num
            if  video_frame_bboxes[frame_num][obj].all()==0:
                video_frame_bboxes[frame_num][obj]=video_frame_bboxes[lastseen_bboxes][obj]
                lastseen_bboxes=frame_num

#             video_frame_bboxes[frame_num][obj][0] += (video_frame_bboxes[frame_num][obj][2]/2.0) # convert x,y to centroid 
#             video_frame_bboxes[frame_num][obj][1] += (video_frame_bboxes[frame_num][obj][3]/2.0)

            yoloBB = video_frame_bboxes[frame_num][obj].tolist() #frame_num, obj1,4 bbox attributes
            yolo_output_temp = yolo_output_temp + yoloBB
        yolo_output_batched = yolo_output_batched + yolo_output_temp
    yolo_output_batched = np.reshape(yolo_output_batched, [batch_size,4])
    return yolo_output_batched

In [191]:

answer_new = load_rolo_gt(0,0,bb[100:150],2,6,0)

In [198]:
answer_new[0]

array([ 243.,  130.,  465.,  483.])

In [7]:
vidmix1 = np.load('model_mixed_dataset_finetune/videomixprediction.npy')
vidmix2 = np.load('model_mixed_dataset_finetune/videomixprediction1.npy')

In [8]:
vidmix1.shape

(775, 4)

In [9]:
vidmix2.shape

(775, 4)

In [1]:
import numpy as np

In [2]:
at= np.ones(shape=(1,6,259))

In [3]:
at.shape

(1, 6, 259)

In [12]:
at[0][-1,:].shape

(259,)

# For Ground Truth
### [camera, ID, frame, left, top, width, height, worldX, worldY, feetX, feetyY]


In [2]:
from scipy.io import loadmat
from shapely.geometry import MultiPoint,box

In [3]:
gt = loadmat('../mat_files/ground_truth/trainval.mat')
gt

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Feb 03 09:27:40 2017',
 '__version__': '1.0',
 'trainData': array([[1.00000000e+00, 2.00000000e+00, 4.41580000e+04, ...,
         7.70919957e+01, 5.08412312e-01, 5.42027327e-01],
        [1.00000000e+00, 2.00000000e+00, 4.41590000e+04, ...,
         7.70918454e+01, 5.08412863e-01, 5.42033834e-01],
        [1.00000000e+00, 2.00000000e+00, 4.41600000e+04, ...,
         7.70916951e+01, 5.08413413e-01, 5.42040340e-01],
        ...,
        [8.00000000e+00, 7.13800000e+03, 1.07036000e+05, ...,
         3.73487261e+01, 6.23263158e-02, 9.66668421e-01],
        [8.00000000e+00, 7.13800000e+03, 1.07037000e+05, ...,
         3.73481854e+01, 6.11192982e-02, 9.67903509e-01],
        [8.00000000e+00, 7.13800000e+03, 1.07038000e+05, ...,
         3.73476447e+01, 5.99122807e-02, 9.69138596e-01]])}

In [4]:
(gt['trainData']).shape

(4077132, 11)

In [5]:
gt = gt['trainData']

In [6]:
gt.shape

(4077132, 11)

# For OpenPose detections
###  [camera, frame, kpx1, kpy1, kpc1, ..., kpx18, kpy18, kpc18]

In [7]:
import numpy as np
import h5py 
f = h5py.File('../mat_files/detections/OpenPose/camera3.mat','r') 

In [8]:
opose_det = f.get('detections')
opose_det = np.array(opose_det)
opose_det= np.transpose(opose_det)
opose_det.shape

(1441094, 56)

In [9]:
#for i in range(100):
print(opose_det[0][:])
# unique, counts = np.unique((a[1][:]), return_counts=True)
# new_dict=(dict(zip(unique, counts)))



[3.       1.       0.941718 0.369694 0.229671 0.943295 0.390221 0.44473
 0.930185 0.392083 0.505429 0.92966  0.435041 0.186249 0.       0.
 0.       0.956435 0.385585 0.325262 0.       0.       0.       0.
 0.       0.       0.933328 0.457466 0.194826 0.933846 0.496678 0.104683
 0.92964  0.550813 0.151536 0.947502 0.46119  0.170145 0.938587 0.499479
 0.166706 0.930696 0.551754 0.240689 0.938572 0.366885 0.159583 0.944345
 0.361277 0.140534 0.93071  0.357573 0.212644 0.951173 0.351019 0.259268]


### GT:               [camera, ID, frame, left, top, width, height, worldX, worldY, feetX, feetyY]
                (4077132, 11)

### Open pose: [camera, frame, kpx1, kpy1, kpc1, ..., kpx18, kpy18, kpc18]
                (56, 1824211)

In [10]:
import pandas as pd

In [11]:
data = gt
df_gt = pd.DataFrame(data,columns=['Camera','ID','frame','topX','topY','wid','ht','wX','wY','fX','fY'])
print (df_gt)

         Camera      ID     frame   topX   topY    wid     ht         wX  \
0           1.0     2.0   44158.0  933.0  378.0   86.0  219.0  22.311128   
1           1.0     2.0   44159.0  933.0  378.0   86.0  219.0  22.310848   
2           1.0     2.0   44160.0  933.0  378.0   86.0  219.0  22.310569   
3           1.0     2.0   44161.0  933.0  378.0   86.0  219.0  22.310290   
4           1.0     2.0   44162.0  933.0  378.0   86.0  219.0  22.310010   
5           1.0     2.0   44163.0  933.0  378.0   86.0  219.0  22.309731   
6           1.0     2.0   44164.0  933.0  378.0   86.0  219.0  22.309452   
7           1.0     2.0   44165.0  933.0  378.0   86.0  219.0  22.309172   
8           1.0     2.0   44166.0  933.0  378.0   86.0  219.0  22.308893   
9           1.0     2.0   44167.0  933.0  378.0   86.0  219.0  22.308614   
10          1.0     2.0   44168.0  933.0  378.0   86.0  219.0  22.308334   
11          1.0     2.0   44169.0  933.0  378.0   86.0  219.0  22.308055   
12          

In [12]:
del df_gt['wX']
del df_gt['wY']
del df_gt['fX']
del df_gt['fY']

In [13]:
 df_gt
#from pprint import pprint

,Camera,ID,frame,topX,topY,wid,ht
0,1.0,2.0,44158.0,933.0,378.0,86.0,219.0
1,1.0,2.0,44159.0,933.0,378.0,86.0,219.0
2,1.0,2.0,44160.0,933.0,378.0,86.0,219.0
3,1.0,2.0,44161.0,933.0,378.0,86.0,219.0
4,1.0,2.0,44162.0,933.0,378.0,86.0,219.0
5,1.0,2.0,44163.0,933.0,378.0,86.0,219.0
6,1.0,2.0,44164.0,933.0,378.0,86.0,219.0
7,1.0,2.0,44165.0,933.0,378.0,86.0,219.0
8,1.0,2.0,44166.0,933.0,378.0,86.0,219.0
9,1.0,2.0,44167.0,933.0,378.0,86.0,219.0


In [69]:
grouped = df_gt.groupby('Camera')
print(type(df_gt))

cam_id = 3.0
obj_id = 251.0

gt_dataframe=grouped.get_group(cam_id).groupby('ID').get_group(obj_id)
print(len(gt_dataframe))
(gt_dataframe)


<class 'pandas.core.frame.DataFrame'>
591


,Camera,ID,frame,topX,topY,wid,ht
1402856,3.0,251.0,71565.0,12.0,446.0,112.0,284.0
1402857,3.0,251.0,71566.0,16.0,446.0,112.0,284.0
1402858,3.0,251.0,71567.0,20.0,446.0,112.0,284.0
1402859,3.0,251.0,71568.0,23.0,446.0,112.0,284.0
1402860,3.0,251.0,71569.0,27.0,446.0,112.0,284.0
1402861,3.0,251.0,71570.0,31.0,446.0,112.0,284.0
1402862,3.0,251.0,71571.0,35.0,446.0,112.0,284.0
1402863,3.0,251.0,71572.0,39.0,446.0,112.0,284.0
1402864,3.0,251.0,71573.0,43.0,446.0,112.0,283.0
1402865,3.0,251.0,71574.0,47.0,447.0,112.0,283.0


In [63]:
import random
a = gt_dataframe['ID'].unique()
bc = random.sample(list(a),5)
bc

[304.0, 143.0, 1109.0, 3253.0, 780.0]

In [45]:
opendata = opose_det #Whole detection 
openpose_dataframe = pd.DataFrame(opendata,columns=['camera', 'frame', 'kpx1', 'kpy1', 'kpc1','kpx2', 'kpy2', 'kpc2','kpx3', 'kpy3', 'kpc3','kpx4', 'kpy4', 'kpc4','kpx5', 'kpy5', 'kpc5','kpx6', 'kpy6', 'kpc6','kpx7', 'kpy7', 'kpc7','kpx8', 'kpy8', 'kpc8','kpx9', 'kpy9', 'kpc9','kpx10', 'kpy10', 'kpc10','kpx11', 'kpy11', 'kpc11','kpx12', 'kpy12', 'kpc12','kpx13', 'kpy13', 'kpc13','kpx14', 'kpy14', 'kpc14','kpx15', 'kpy15', 'kpc15','kpx16', 'kpy16', 'kpc16','kpx17', 'kpy17', 'kpc17', 'kpx18', 'kpy18', 'kpc18'])

op_dataframe1=openpose_dataframe.groupby('frame').get_group(54815.0)
op_dataframe1

,camera,frame,kpx1,kpy1,kpc1,kpx2,kpy2,kpc2,kpx3,kpy3,...,kpc15,kpx16,kpy16,kpc16,kpx17,kpy17,kpc17,kpx18,kpy18,kpc18
181620,3.0,54815.0,0.000000,0.000000,0.000000,0.035160,0.449049,0.674779,0.039892,0.451842,...,0.000000,0.000000,0.000000,0.000000,0.049863,0.421044,0.467770,0.036753,0.413557,0.148636
181621,3.0,54815.0,0.934903,0.351955,0.504568,0.942268,0.390223,0.439505,0.929643,0.390252,...,0.541445,0.939096,0.340772,0.518016,0.926003,0.352890,0.197068,0.948025,0.345439,0.289002
181622,3.0,54815.0,0.089752,0.301521,0.126916,0.089229,0.311807,0.579515,0.081873,0.311814,...,0.116014,0.090301,0.295965,0.116526,0.087137,0.295975,0.068771,0.092908,0.298751,0.094430
181623,3.0,54815.0,0.000000,0.000000,0.000000,0.250395,0.292215,0.320197,0.239901,0.287539,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [46]:
output =[]
output_gt = []
for row_index_gt,row_gt in gt_dataframe.iterrows(): #[Camera,ID,frame,topX,topY,wid,ht]
    op_dataframe=openpose_dataframe.groupby('frame').get_group(row_gt[2])
    bbox = box((row_gt[3])/1920.0,(row_gt[4])/1080.0,(row_gt[3]+row_gt[5])/1920.0,(row_gt[4]+row_gt[6])/1080.0)
    #print (row_gt[2])
    for row_index_op,row_op in op_dataframe.iterrows(): #[camera,frame,kpx1,kpy1,kpc1,kpx2,kpy2,kpc2,kpx3,kpy3,...,kpc15,kpx16,kpy16,kpc16,kpx17,kpy17,kpc17,kpx18,kpy18,kpc18]
        
        kpoints = MultiPoint([
            (row_op[2],row_op[3]),
            (row_op[5],row_op[6]),
            (row_op[8],row_op[9]),
            (row_op[11],row_op[12]),
            (row_op[14],row_op[15]),
            (row_op[17],row_op[18]),
            (row_op[20],row_op[21]),
            (row_op[23],row_op[24]),
            (row_op[26],row_op[27]),
            (row_op[29],row_op[30]),
            (row_op[32],row_op[33]),
            (row_op[35],row_op[36]),
            (row_op[38],row_op[39]),
            (row_op[41],row_op[42]),
            (row_op[44],row_op[45]),
            (row_op[47],row_op[48]),
            (row_op[50],row_op[51]),
            (row_op[53],row_op[54]),
        ])
        cnt=0
        for i in range(18):
            if bbox.contains(kpoints[i]):
                cnt =cnt+1
        if cnt>8:
            #print (row_gt)
            output.append(pd.np.array(row_op[2:],dtype='float64'))
            output_gt.append(pd.np.array(row_gt[3:],dtype='float64'))
            break
            



In [47]:
final = np.asarray(output,dtype=np.float64)
final =np.reshape(final,newshape=(final.shape[0],1,final.shape[1]))

final_gt = np.asarray(output_gt,dtype=np.float64)
final_gt =np.reshape(final_gt,newshape=(final_gt.shape[0],1,final_gt.shape[1]))

#op_dataframe[181894]
#op_dataframe[181894]

In [172]:
print(final.shape,final_gt.shape)
import pprint

(512, 1, 54) (512, 1, 4)


In [42]:
np.save('duke_dataset/training/'+'cam'+str(cam_id)[:-2]+'obj'+str(obj_id)[:-2],final)
np.save('duke_dataset/training/'+'cam'+str(cam_id)[:-2]+'obj'+str(obj_id)[:-2]+'_gt',final_gt)


In [178]:
for i in range(20):
    print(final[i,:,:7])

[[0.        0.        0.        0.0351604 0.449049  0.674779  0.0398921]]
[[0.0577491 0.428502  0.0723873 0.0362233 0.448121  0.650624  0.0419989]]
[[0.        0.        0.        0.0393875 0.448106  0.661487  0.0446313]]
[[0.        0.        0.        0.0404265 0.446251  0.624973  0.0472329]]
[[0.        0.        0.        0.0440838 0.449966  0.635521  0.0488157]]
[[0.        0.        0.        0.0467126 0.447188  0.605213  0.0519514]]
[[0.0693051 0.423852  0.067081  0.0488136 0.445312  0.609093  0.0535443]]
[[0.0729493 0.421969  0.124562  0.0514426 0.445349  0.622993  0.0561667]]
[[0.        0.        0.        0.0525181 0.449045  0.65293   0.0598276]]
[[0.0776693 0.421986  0.1314    0.0561552 0.449044  0.647347  0.0608829]]
[[0.0808293 0.427598  0.0506343 0.0572353 0.44811   0.579755  0.0624639]]
[[0.0813779 0.424802  0.0625357 0.0598409 0.449975  0.662894  0.0650952]]
[[0.0839906 0.428519  0.100243  0.0629907 0.449052  0.604749  0.067195 ]]
[[0.0855744 0.426659  0.0928086 0.0645

In [37]:
for row_index_op,row_op in op_dataframe.iterrows(): 
    #[camera,frame,kpx1,kpy1,kpc1,kpx2,kpy2,kpc2,kpx3,kpy3,...,kpc15,kpx16,kpy16,kpc16,kpx17,kpy17,kpc17,kpx18,kpy18,kpc18]
    print (type(row_op))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [74]:
a=41.0
print(str(a)[:-2])

41


In [205]:
a = np.load('duke_dataset/training/cam1obj790.npy')

In [206]:
a.shape

(627, 1, 54)

In [98]:
a

array([[[0.324923 , 0.364117 , 0.859078 , ..., 0.328608 , 0.361289 ,
         0.201032 ]],

       [[0.322831 , 0.362218 , 0.819917 , ..., 0.326002 , 0.35945  ,
         0.423183 ]],

       [[0.321285 , 0.362265 , 0.857814 , ..., 0.325486 , 0.360341 ,
         0.408318 ]],

       ...,

       [[0.109184 , 0.390229 , 0.52061  , ..., 0.116007 , 0.391173 ,
         0.423426 ]],

       [[0.       , 0.       , 0.       , ..., 0.103407 , 0.489205 ,
         0.0598137]],

       [[0.107602 , 0.390246 , 0.588782 , ..., 0.114959 , 0.385605 ,
         0.513406 ]]])

In [70]:
import os
!pwd

/home/pkulkar7/object_tracking


In [75]:
paths = sorted(os.listdir('duke_dataset/training/'))

In [120]:
len(paths)

128

In [85]:
len(paths)

128

In [100]:
for i in range(int(len(paths)/2)):
    i=i*2
    print(paths[i])
    print(i)
    

cam1obj18.npy
0
cam1obj19.npy
2
cam1obj20.npy
4
cam1obj21.npy
6
cam1obj30.npy
8
cam1obj31.npy
10
cam1obj600.npy
12
cam1obj790.npy
14
cam2obj109.npy
16
cam2obj18.npy
18
cam2obj202.npy
20
cam2obj50.npy
22
cam2obj51.npy
24
cam2obj740.npy
26
cam2obj790.npy
28
cam2obj90.npy
30
cam3obj162.npy
32
cam3obj1693.npy
34
cam3obj182.npy
36
cam3obj251.npy
38
cam3obj47.npy
40
cam3obj516.npy
42
cam3obj698.npy
44
cam3obj731.npy
46
cam4obj1636.npy
48
cam4obj1745.npy
50
cam4obj1749.npy
52
cam4obj2044.npy
54
cam4obj250.npy
56
cam4obj371.npy
58
cam4obj595.npy
60
cam4obj99.npy
62
cam5obj1587.npy
64
cam5obj1766.npy
66
cam5obj189.npy
68
cam5obj205.npy
70
cam5obj332.npy
72
cam5obj590.npy
74
cam5obj766.npy
76
cam5obj98.npy
78
cam6obj2471.npy
80
cam6obj29.npy
82
cam6obj343.npy
84
cam6obj346.npy
86
cam6obj4243.npy
88
cam6obj444.npy
90
cam6obj4494.npy
92
cam6obj4550.npy
94
cam7obj159.npy
96
cam7obj4096.npy
98
cam7obj4107.npy
100
cam7obj4431.npy
102
cam7obj4640.npy
104
cam7obj4694.npy
106
cam7obj5333.npy
108
cam7obj

In [79]:
b

2

In [101]:
import torch

In [121]:
a=np.load('duke_dataset/training/cam1obj19.npy')

In [122]:
a.shape

(750, 1, 54)

In [115]:
a = torch.from_numpy(a)

In [117]:
a.shape

torch.Size([627, 1, 54])

In [162]:
import random

In [163]:
random.seed(0)

In [168]:
ne=random.sample(range(0,128,2),64)
ne

[66,
 44,
 114,
 92,
 60,
 106,
 122,
 72,
 20,
 88,
 86,
 26,
 98,
 6,
 116,
 110,
 96,
 42,
 126,
 32,
 14,
 76,
 56,
 22,
 0,
 118,
 52,
 112,
 64,
 38,
 124,
 48,
 108,
 8,
 34,
 90,
 78,
 28,
 46,
 4,
 94,
 2,
 58,
 16,
 84,
 102,
 30,
 80,
 18,
 50,
 10,
 120,
 74,
 24,
 70,
 68,
 12,
 104,
 100,
 54,
 40,
 62,
 36,
 82]

In [169]:
# for i in ne:
#     if i%2!=0:
#         print (i)
#     else:
#         print ("Success")
dict((x,ne.count(ne)) for x in set(ne))
        

{0: 0,
 2: 0,
 4: 0,
 6: 0,
 8: 0,
 10: 0,
 12: 0,
 14: 0,
 16: 0,
 18: 0,
 20: 0,
 22: 0,
 24: 0,
 26: 0,
 28: 0,
 30: 0,
 32: 0,
 34: 0,
 36: 0,
 38: 0,
 40: 0,
 42: 0,
 44: 0,
 46: 0,
 48: 0,
 50: 0,
 52: 0,
 54: 0,
 56: 0,
 58: 0,
 60: 0,
 62: 0,
 64: 0,
 66: 0,
 68: 0,
 70: 0,
 72: 0,
 74: 0,
 76: 0,
 78: 0,
 80: 0,
 82: 0,
 84: 0,
 86: 0,
 88: 0,
 90: 0,
 92: 0,
 94: 0,
 96: 0,
 98: 0,
 100: 0,
 102: 0,
 104: 0,
 106: 0,
 108: 0,
 110: 0,
 112: 0,
 114: 0,
 116: 0,
 118: 0,
 120: 0,
 122: 0,
 124: 0,
 126: 0}